In [112]:
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_ollama import ChatOllama
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import RetrievalQA  # <-- Ganti ke RetrievalQA
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
import json
from langchain.schema import Document
from langchain_google_genai import ChatGoogleGenerativeAI



In [ ]:
import json
from langchain.schema import Document

documents = []

# ----------------------------
# 1. Proses file formasi CPNS
# ----------------------------
cpns_file = '/Users/muhammadzuamaalamin/Documents/risetmandiir/project/AIRA/helpsekfix/data/data.json'

with open(cpns_file, 'r', encoding='utf-8') as f:
    raw_cpns = json.load(f)

# Pastikan berupa list
if isinstance(raw_cpns, dict):
    raw_cpns = [raw_cpns]
elif not isinstance(raw_cpns, list):
    raise ValueError("File formasi harus berupa object atau array of object.")

# ----------------------------
# 2. Konversi ke Document
# ----------------------------
for idx, item in enumerate(raw_cpns):
    # Gabungkan semua field penting ke dalam teks (content)
    # Format teks dibuat agar mudah dipahami oleh model RAG
    text_parts = [
        f"Jabatan: {item.get('jabatan', 'Tidak disebutkan')}",
        f"Instansi: {item.get('instansi', 'Tidak disebutkan')}",
        f"Unit Kerja: {item.get('unit_kerja', 'Tidak disebutkan')}",
        f"Penempatan: {item.get('penempatan', 'Tidak disebutkan')}",
    ]
    
    if 'catatan_penempatan' in item:
        text_parts.append(f"Catatan Penempatan: {item['catatan_penempatan']}")
    
    text_parts.extend([
        f"Jenis Formasi: {item.get('jenis_formasi', 'Tidak disebutkan')}",
        f"Khusus Disabilitas: {'Ya' if item.get('khusus_disabilitas', False) else 'Tidak'}",
        f"Penghasilan (juta): {item['penghasilan']['min']} - {item['penghasilan']['max']}",
        f"Jumlah Kebutuhan: {item.get('jumlah_kebutuhan', 0)}",
    ])

    # Format kualifikasi pendidikan (list)
    pendidikan = item.get('kualifikasi_pendidikan', [])
    if isinstance(pendidikan, str):
        pendidikan = [pendidikan]
    pendidikan_str = "; ".join(pendidikan)
    text_parts.append(f"Kualifikasi Pendidikan: {pendidikan_str}")

    if 'website_instansi' in item:
        text_parts.append(f"Website Instansi: {item['website_instansi']}")

    # Gabung jadi satu teks
    page_content = "\n".join(text_parts)

    # Metadata untuk filtering cepat di RAG
    metadata = {
        "jabatan": item.get("jabatan"),
        "instansi": item.get("instansi"),
        "jenis_formasi": item.get("jenis_formasi"),
        "khusus_disabilitas": item.get("khusus_disabilitas", False),
        "penempatan": item.get("penempatan"),
        "min_gaji": item["penghasilan"]["min"],
        "max_gaji": item["penghasilan"]["max"],
        "jumlah_kebutuhan": item.get("jumlah_kebutuhan", 0),
        "sumber_file": cpns_file,
        "index": idx
    }

    # Buat Document
    doc = Document(page_content=page_content, metadata=metadata)
    documents.append(doc)

# ----------------------------
# 1b. Proses file FAQ ASN
# ----------------------------
faq_file = '/Users/muhammadzuamaalamin/Documents/risetmandiir/project/AIRA/helpsekfix/data/faq.json'

with open(faq_file, 'r', encoding='utf-8') as f:
    raw_faq = json.load(f)

faq_docs = []
for idx, item in enumerate(raw_faq):
    text = (
        f"[FAQ ASN]\n"
        f"Pertanyaan: {item.get('Pertanyaan (FAQ)', 'Tidak disebutkan')}\n"
        f"Jawaban: {item.get('Jawaban', 'Tidak disebutkan')}\n"
        f"Regulasi: {item.get('Regulasi yang Menjadi Dasar', '-')}\n"
        f"Sumber: {item.get('Sumber', '-')}"
    )

    metadata = {
        "tipe": "faq",
        "no": item.get("No", idx),
        "sumber": item.get("Sumber", ""),
    }

    faq_docs.append(Document(page_content=text, metadata=metadata))

In [114]:
documents
len(documents)

30

In [115]:
## Embedding
# 3) Siapkan embedding (sekali saja)
# embeddings = HuggingFaceEmbeddings(
#     model_name="/Users/muhammadzuamaalamin/Documents/fintunellm/model/multilingual-e5-large-instruct",
#     model_kwargs={"device": "cpu"}   # ganti "cuda" / "mps" bila tersedia dan ingin pakai GPU/MPS
# )
embeddings = HuggingFaceEmbeddings(model_name="/Users/muhammadzuamaalamin/Documents/fintunellm/model/bge-m3",
                                   model_kwargs={"device": "cpu"})

In [ ]:

# 4) Buat FAISS index dari CHUNKS (penting: gunakan chunks)
db = FAISS.from_documents(documents, embeddings)
db.save_local("faiss_index123")
del db  # hapus dari memori

# nanti kalau butuh lagi
db = FAISS.load_local("faiss_index123", embeddings, allow_dangerous_deserialization=True)


In [117]:
# 5) Pencarian mirip/kemiripan
query = "s1 hukum"
# Jika tersedia, gunakan method yang mengembalikan score
try:
    results_with_score = db.similarity_search_with_score(query, k=3)
    for idx, (doc, score) in enumerate(results_with_score, 1):
        print(f"Result {idx} — score={score:.4f}\n{doc.page_content[:800]}\n")
except AttributeError:
    # fallback bila method with_score tidak ada
    results = db.similarity_search(query, k=3)
    for idx, doc in enumerate(results, 1):
        print(f"Result {idx}\n{doc.page_content[:800]}\n")


Result 1 — score=1.4010
Pertanyaan: Apakah diperbolehkan memilih formasi yang lebih rendah dari ijazah yang saya miliki (misal: saya memiliki ijazah S2 untuk melamar formasi S1)?
Jawaban: Tidak diperbolehkan. Pelamar hanya dapat melamar pada formasi yang sesuai dengan kualifikasi pendidikan yang tercantum pada ijazah.
Dasar Regulasi: Permen PANRB Nomor 6 Tahun 2024 Pasal 9 ayat (1)
Sumber: https://sscasn.bkn.go.id/faqpns

Result 2 — score=1.4034
Jabatan: ARSIPARIS AHLI PERTAMA
Instansi: Kejaksaan Agung
Unit Kerja: Kejaksaan Agung
Penempatan: Nasional
Catatan Penempatan: Lokasi mengikuti satuan kerja yang dipilih di SSCASN (Kejaksaan Agung, Kejati, atau Kejari di seluruh Indonesia).
Jenis Formasi: CPNS Putra/putri Papua Dan Papua Barat
Khusus Disabilitas: Tidak
Penghasilan (juta): 8.54 - 10.53
Jumlah Kebutuhan: 17
Kualifikasi Pendidikan: D4 PENGELOLAAN ARSIP DAN REKAMAN INFORMASI; D4 MANAJEMEN REKOD DAN ARSIP; D4 KEARSIPAN; D4 KEARSIPAN DIGITAL; S1 ILMU LINGKUNGAN; S1 ANTROPOLOGI; S1 AD

In [ ]:

# --- Setup LLM (tanpa memory) ---
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    google_api_key="XXXXXXXXXXXXXXX",
    temperature=0.2
)
# Buat retriever
retriever = db.as_retriever(search_kwargs={"k": 3})

# --- Output Parser ---
schemas = [
    ResponseSchema(
        name="jawaban",
        description="Jawaban lengkap, jelas, dan sesuai instruksi dalam bahasa Indonesia yang mudah dipahami."
    )
]
output_parser = StructuredOutputParser.from_response_schemas(schemas)
format_instructions = output_parser.get_format_instructions()

In [ ]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template("""
Kamu adalah asisten resmi dari Badan Kepegawaian Negara (BKN) yang membantu masyarakat memahami informasi seputar Aparatur Sipil Negara (ASN), termasuk CPNS, PPPK, formasi, kualifikasi pendidikan, alur pendaftaran, dan regulasi terkait.

Instruksi:
1. Jawab PERTANYAAN PENGGUNA secara LENGKAP namun LANGSUNG KE INTI.
   - Jika pertanyaan tentang formasi: sebutkan jabatan, instansi, jumlah kebutuhan, kualifikasi pendidikan, dan kisaran gaji (jika tersedia dalam konteks).
   - Jika pertanyaan tentang konsep (misal: “Apa itu CPNS?”): berikan penjelasan sederhana dalam bahasa Indonesia yang mudah dipahami.

2. Penafsiran kualifikasi:
   - “S1 semua jurusan” berarti semua lulusan S1 dari jurusan apa pun boleh mendaftar.
   - Pelamar S1 hanya boleh mendaftar ke formasi yang mensyaratkan kualifikasi S1.
   - Pelamar S2 hanya boleh mendaftar ke formasi yang mensyaratkan kualifikasi S2.
   - Pelamar S3 hanya boleh mendaftar ke formasi yang mensyaratkan kualifikasi S3.
   - Jangan merekomendasikan pelamar ke formasi dengan kualifikasi di bawah atau di atas jenjang pendidikannya.

3. Jika ada banyak formasi yang relevan:
   - Tampilkan SEMUA formasi yang sesuai.
   - Setiap formasi ditulis dalam satu baris terpisah.
   - Urutkan berdasarkan: (a) jumlah kebutuhan (terbanyak → paling sedikit), lalu (b) kisaran gaji (jika tersedia).
   - Formasi dengan jumlah kebutuhan lebih tinggi dianggap memiliki peluang lolos lebih besar.

4. Gunakan HANYA informasi yang tersedia dalam KONTEKS.
   - Jangan mengarang, menebak, atau menambahkan data di luar konteks.

5. Jika konteks tidak mencukupi:
   - Untuk topik ASN/CPNS/PPPK:  
     → "Maaf, saya tidak tahu jawaban pastinya berdasarkan dokumen yang tersedia."
   - Untuk topik di luar rekrutmen ASN:  
     → "Maaf, saya tidak tahu. Saya hanya bisa menjawab pertanyaan seputar formasi CPNS, kualifikasi pendidikan, dan informasi rekrutmen ASN berdasarkan dokumen resmi."

6. Selalu akhiri dengan arahan:  
   → "Untuk informasi lebih lengkap dan terkini, silakan kunjungi https://sscasn.bkn.go.id"

Pertanyaan pengguna: {question}
Konteks tambahan dari dokumen:
{context}
{format_instructions}
""")

In [120]:
# --- Buat Chain tanpa memory ---
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type_kwargs={"prompt": prompt_template.partial(format_instructions=format_instructions)},
    return_source_documents=False  # opsional
)


In [121]:
# --- Eksekusi Query ---
query = "formasi yang cocok untuk lulusan s3 ilmu komputer ?"
response = qa_chain.invoke({"query": query})  # Pastikan key-nya sesuai schema chain kamu

# --- Parsing Output ---
try:
    # LangChain RetrievalQA biasanya mengembalikan dict dgn key 'result' atau 'answer'
    if isinstance(response, dict):
        raw_answer = response.get("result") or response.get("answer") or str(response)
    else:
        raw_answer = str(response)  # fallback kalau response langsung string

    # Coba parse sesuai format yang kamu buat di OutputParser
    parsed = output_parser.parse(raw_answer)
    jawaban = parsed.get("jawaban", raw_answer)

except Exception as e:
    print("⚠️ Gagal parsing:", e)
    print("Raw output:", raw_answer)
    jawaban = f"Maaf, terjadi kesalahan saat memproses jawaban.\nDetail: {e}"

# --- Output Akhir ---
print("\n==============================")
print("💬 Jawaban Asisten BKN:")
print(jawaban.strip())
print("==============================\n")



💬 Jawaban Asisten BKN:
Maaf, saya tidak menemukan informasi formasi yang spesifik untuk lulusan S3 Ilmu Komputer dalam dokumen yang tersedia.

Untuk informasi lebih lengkap dan terkini, silakan kunjungi https://sscasn.bkn.go.id



In [122]:
# --- Eksekusi Query ---
query = "apa itu cpns atau asn? ?"
response = qa_chain.invoke({"query": query})  # <-- perhatikan: key-nya "query", bukan "question"

# --- Parsing Output ---
try:
    raw_answer = response["result"]  # RetrievalQA mengembalikan {"result": "..."}
    parsed = output_parser.parse(raw_answer)
    jawaban = parsed.get("jawaban", "Maaf, saya tidak dapat memberikan jawaban.")
except Exception as e:
    print("⚠️ Gagal parsing:", e)
    print("Raw output:", raw_answer)
    jawaban = "Maaf, terjadi kesalahan saat memproses jawaban."

print("\n==============================")
print("💬 Jawaban Asisten BKN:")
print(jawaban)
print("==============================\n")


💬 Jawaban Asisten BKN:
ASN (Aparatur Sipil Negara) terdiri dari PNS dan PPPK yang diangkat untuk menjalankan tugas pemerintahan dan pelayanan publik berdasarkan sistem merit, profesionalisme, dan kompetensi.

📘 Dasar: Undang-Undang Nomor 20 Tahun 2023 tentang Aparatur Sipil Negara
🔗 Sumber rujukan: https://peraturan.bpk.go.id/Details/269470/uu-no-20-tahun-2023

Untuk informasi lebih lengkap dan terkini, silakan kunjungi https://sscasn.bkn.go.id



In [97]:
# --- Eksekusi Query ---
query = "formasi dengan gaji tertinggi apa ?"
response = qa_chain.invoke({"query": query})  # Pastikan key-nya sesuai schema chain kamu

# --- Parsing Output ---
try:
    # LangChain RetrievalQA biasanya mengembalikan dict dgn key 'result' atau 'answer'
    if isinstance(response, dict):
        raw_answer = response.get("result") or response.get("answer") or str(response)
    else:
        raw_answer = str(response)  # fallback kalau response langsung string

    # Coba parse sesuai format yang kamu buat di OutputParser
    parsed = output_parser.parse(raw_answer)
    jawaban = parsed.get("jawaban", raw_answer)

except Exception as e:
    print("⚠️ Gagal parsing:", e)
    print("Raw output:", raw_answer)
    jawaban = f"Maaf, terjadi kesalahan saat memproses jawaban.\nDetail: {e}"

# --- Output Akhir ---
print("\n==============================")
print("💬 Jawaban Asisten BKN:")
print(jawaban.strip())
print("==============================\n")



💬 Jawaban Asisten BKN:
Berdasarkan informasi yang tersedia, formasi dengan kisaran gaji tertinggi adalah:

*   **Jabatan:** ARSIPARIS AHLI PERTAMA
    **Instansi:** Kejaksaan Agung
    **Jenis Formasi:** CPNS Umum
    **Jumlah Kebutuhan:** 484
    **Kualifikasi Pendidikan:** D4 PENGELOLAAN ARSIP DAN REKAMAN INFORMASI; D4 MANAJEMEN REKOD DAN ARSIP; D4 KEARSIPAN; D4 KEARSIPAN DIGITAL; S1 ILMU LINGKUNGAN; S1 ANTROPOLOGI; S1 ADMINISTRASI NEGARA; S1 KEHUTANAN; S1 EKONOMI; S1 PENDIDIKAN AGAMA ISLAM; S1 PENDIDIKAN KEAGAMAAN KATOLIK; S1 PENDIDIKAN KEAGAMAAN BUDDHA; S1 ILMU KELAUTAN; S1 PENDIDIKAN KEAGAMAAN HINDU; S1 PENDIDIKAN KEAGAMAAN KRISTEN; S1 SEJARAH; S1 ILMU SEJARAH; S1 PSIKOLOGI; S1 SOSIOLOGI; S1 ILMU PERPUSTAKAAN; S1 KEARSIPAN; S1 PENDIDIKAN ILMU PENGETAHUAN SOSIAL; S1 ILMU PERTANIAN; S1 HUBUNGAN MASYARAKAT
    **Kisaran Gaji:** 8.54 - 10.53 juta

*   **Jabatan:** ARSIPARIS AHLI PERTAMA
    **Instansi:** Kejaksaan Agung
    **Jenis Formasi:** CPNS Penyandang Disabilitas
    **Jumlah 

In [98]:
# --- Eksekusi Query ---
query = "saya merupakan lulusan S3 kira kira formasi apa yang cocok untuk saya  ?"
response = qa_chain.invoke({"query": query})  # <-- perhatikan: key-nya "query", bukan "question"

# --- Parsing Output ---
try:
    raw_answer = response["result"]  # RetrievalQA mengembalikan {"result": "..."}
    parsed = output_parser.parse(raw_answer)
    jawaban = parsed.get("jawaban", "Maaf, saya tidak dapat memberikan jawaban.")
except Exception as e:
    print("⚠️ Gagal parsing:", e)
    print("Raw output:", raw_answer)
    jawaban = "Maaf, terjadi kesalahan saat memproses jawaban."

print("\n==============================")
print("💬 Jawaban Asisten BKN:")
print(jawaban)
print("==============================\n")


💬 Jawaban Asisten BKN:
Berdasarkan kualifikasi pendidikan S3 Anda, berikut adalah formasi yang cocok:

Jabatan: PENELITI AHLI MUDA
Instansi: Badan Riset dan Inovasi Nasional
Jumlah Kebutuhan: 75
Kualifikasi Pendidikan: S3 semua jurusan; S3 Ilmu Komputer; S3 Teknik Informatika; S3 Sistem Informasi; S3 Statistika; S3 Matematika; S3 Fisika; S3 Kimia; S3 Biologi; S3 Kesehatan Masyarakat; S3 Ilmu Ekonomi; S3 Manajemen; S3 Ilmu Pemerintahan; S3 Sosiologi; S3 Antropologi; S3 Ilmu Lingkungan; S3 Teknik Elektro; S3 Agronomi; S3 Farmasi; S3 Ilmu Pendidikan
Kisaran Gaji: 7 - 11 juta

Untuk informasi lebih lengkap dan terkini, silakan kunjungi https://sscasn.bkn.go.id



In [23]:
# --- Eksekusi Query ---
query = "saya merupakan lulusan s3 fisika formasi mana yang memiliki pelunag lolos paling tinggi ?"
response = qa_chain.invoke({"query": query})  # <-- perhatikan: key-nya "query", bukan "question"

# --- Parsing Output ---
try:
    raw_answer = response["result"]  # RetrievalQA mengembalikan {"result": "..."}
    parsed = output_parser.parse(raw_answer)
    jawaban = parsed.get("jawaban", "Maaf, saya tidak dapat memberikan jawaban.")
except Exception as e:
    print("⚠️ Gagal parsing:", e)
    print("Raw output:", raw_answer)
    jawaban = "Maaf, terjadi kesalahan saat memproses jawaban."

print("\n==============================")
print("💬 Jawaban Asisten BKN:")
print(jawaban)
print("==============================\n")


💬 Jawaban Asisten BKN:
Sebagai lulusan S3 Fisika, Anda memenuhi kualifikasi untuk beberapa formasi. Berdasarkan jumlah kebutuhan, formasi dengan peluang lolos paling tinggi adalah sebagai berikut:

1. Jabatan: PENELITI AHLI MUDA, Instansi: Badan Riset dan Inovasi Nasional, Jenis Formasi: CPNS Diaspora, Khusus Disabilitas: Ya, Jumlah Kebutuhan: 125, Kualifikasi Pendidikan: S3 semua jurusan; S3 Ilmu Komputer; S3 Teknik Informatika; S3 Sistem Informasi; S3 Statistika; S3 Matematika; S3 Fisika; S3 Kimia; S3 Biologi; S3 Kesehatan Masyarakat; S3 Ilmu Ekonomi; S3 Manajemen; S3 Ilmu Pemerintahan; S3 Sosiologi; S3 Antropologi; S3 Ilmu Lingkungan; S3 Teknik Elektro; S3 Agronomi; S3 Farmasi; S3 Ilmu Pendidikan, Kisaran Gaji: 7 - 11 juta.
2. Jabatan: PENELITI AHLI MUDA, Instansi: Badan Riset dan Inovasi Nasional, Jenis Formasi: CPNS Lulusan Terbaik, Khusus Disabilitas: Ya, Jumlah Kebutuhan: 75, Kualifikasi Pendidikan: S3 semua jurusan; S3 Ilmu Komputer; S3 Teknik Informatika; S3 Sistem Informasi;

In [25]:
# --- Eksekusi Query ---
query = "siapa presiden ri ?"
response = qa_chain.invoke({"query": query})  # <-- perhatikan: key-nya "query", bukan "question"

# --- Parsing Output ---
try:
    raw_answer = response["result"]  # RetrievalQA mengembalikan {"result": "..."}
    parsed = output_parser.parse(raw_answer)
    jawaban = parsed.get("jawaban", "Maaf, saya tidak dapat memberikan jawaban.")
except Exception as e:
    print("⚠️ Gagal parsing:", e)
    print("Raw output:", raw_answer)
    jawaban = "Maaf, terjadi kesalahan saat memproses jawaban."

print("\n==============================")
print("💬 Jawaban Asisten BKN:")
print(jawaban)
print("==============================\n")


💬 Jawaban Asisten BKN:
Maaf, saya tidak tahu. Saya hanya bisa menjawab pertanyaan seputar formasi CPNS, kualifikasi pendidikan, dan informasi rekrutmen ASN berdasarkan dokumen resmi.
Untuk informasi lebih lengkap dan terkini, silakan kunjungi https://sscasn.bkn.go.id



In [1]:
# --- Eksekusi Query ---
query = "berikan data formasi yang penempatnnya di jakarta?"
response = qa_chain.invoke({"query": query})  # <-- perhatikan: key-nya "query", bukan "question"

# --- Parsing Output ---
try:
    raw_answer = response["result"]  # RetrievalQA mengembalikan {"result": "..."}
    parsed = output_parser.parse(raw_answer)
    jawaban = parsed.get("jawaban", "Maaf, saya tidak dapat memberikan jawaban.")
except Exception as e:
    print("⚠️ Gagal parsing:", e)
    print("Raw output:", raw_answer)
    jawaban = "Maaf, terjadi kesalahan saat memproses jawaban."

print("\n==============================")
print("💬 Jawaban Asisten BKN:")
print(jawaban)
print("==============================\n")

NameError: name 'qa_chain' is not defined

In [ ]:
# --- Eksekusi Query ---
query = "berikan data formasi dari BRIN?"
response = qa_chain.invoke({"query": query})  # <-- perhatikan: key-nya "query", bukan "question"

# --- Parsing Output ---
try:
    raw_answer = response["result"]  # RetrievalQA mengembalikan {"result": "..."}
    parsed = output_parser.parse(raw_answer)
    jawaban = parsed.get("jawaban", "Maaf, saya tidak dapat memberikan jawaban.")
except Exception as e:
    print("⚠️ Gagal parsing:", e)
    print("Raw output:", raw_answer)
    jawaban = "Maaf, terjadi kesalahan saat memproses jawaban."

print("\n==============================")
print("💬 Jawaban Asisten BKN:")
print(jawaban)
print("==============================\n")


💬 Jawaban Asisten BKN:
Berikut adalah data formasi dari Badan Riset dan Inovasi Nasional (BRIN):

Peneliti Ahli Muda
Instansi: Badan Riset dan Inovasi Nasional
Jumlah Kebutuhan: 25
Kualifikasi Pendidikan: S3 semua jurusan; S3 Ilmu Komputer; S3 Teknik Informatika; S3 Sistem Informasi; S3 Statistika; S3 Matematika; S3 Fisika; S3 Kimia; S3 Biologi; S3 Kesehatan Masyarakat; S3 Ilmu Ekonomi; S3 Manajemen; S3 Ilmu Pemerintahan; S3 Sosiologi; S3 Antropologi; S3 Ilmu Lingkungan; S3 Teknik Elektro; S3 Agronomi; S3 Farmasi; S3 Ilmu Pendidikan
Penghasilan: 7 - 11 juta

Peneliti Ahli Muda
Instansi: Badan Riset dan Inovasi Nasional
Jumlah Kebutuhan: 75
Kualifikasi Pendidikan: S3 semua jurusan; S3 Ilmu Komputer; S3 Teknik Informatika; S3 Sistem Informasi; S3 Statistika; S3 Matematika; S3 Fisika; S3 Kimia; S3 Biologi; S3 Kesehatan Masyarakat; S3 Ilmu Ekonomi; S3 Manajemen; S3 Ilmu Pemerintahan; S3 Sosiologi; S3 Antropologi; S3 Ilmu Lingkungan; S3 Teknik Elektro; S3 Agronomi; S3 Farmasi; S3 Ilmu Pend